In [4]:
%%sql
create database VehiclesDemo

In [5]:
%%sql

-- Create a Spark table over Cosmos DB analytical store
-- To select a preferred list of regions in a multi-region Cosmos DB account, add spark.cosmos.preferredRegions '<Region1>,<Region2>' in the config options

create table VehiclesDemo.VehiclesLog using cosmos.olap options (
    spark.synapse.linkedService 'CosmosDbTest',
    spark.cosmos.container 'VehiclesLog'
)

In [5]:
%%sql
select count(1) from VehiclesDemo.VehiclesLog

In [6]:
%%sql
SELECT
    COUNT(distinct vehicle_id) as TotalVehicles,
    COUNT(distinct CASE WHEN status IN ('trip-started','trip-otw','trip-finished') THEN vehicle_id ELSE NULL END) as VehiclesOnTheWay
FROM (
    SELECT
        V.vehicle_id,
        V.status
    FROM FROM VehiclesDemo.VehiclesLog V
    INNER JOIN (
        SELECT
            vehicle_id,
            MAX(Timestamp) as max_timestamp
        FROM VehiclesDemo.VehiclesLog
        WHERE timestamp > '2020-07-29 16:00:40.000'
        GROUP BY vehicle_id
    ) V1 ON V.vehicle_id = V1.vehicle_id and v.timestamp = V1.max_timestamp
) Vehicle(vehicle_id,status)


In [4]:
from datetime import datetime
import requests

powerbi_url = 'https://api.powerbi.com/beta/e6816e25-41db-4bb7-b1a6-25a718776f8c/datasets/0e52da13-d7f0-4de4-9c17-0dfae5901086/rows?noSignUpCheck=1&key=KuP07u%2BOUXi307hcRo2fqD6DCC93LaB5aLD5VmcOlKInPJhxfuT7j55ePVgA7I3SVp%2Bh087QfOwzlHF85d69Rw%3D%3D'

df = spark.sql('''
	SELECT
		COUNT(distinct vehicle_id) as TotalVehicles,
		COUNT(distinct CASE WHEN status IN ('trip-started','trip-otw','trip-finished') THEN vehicle_id ELSE NULL END) as VehiclesOnTheWay
	FROM (
		SELECT
			V.vehicle_id,
			V.status
		FROM FROM VehiclesDemo.VehiclesLog V
		INNER JOIN (
			SELECT
				vehicle_id,
				MAX(Timestamp) as max_timestamp
			FROM VehiclesDemo.VehiclesLog
			WHERE timestamp > '2020-07-29 16:00:40.000'
			GROUP BY vehicle_id
		) V1 ON V.vehicle_id = V1.vehicle_id and v.timestamp = V1.max_timestamp
	) Vehicle(vehicle_id,status)
''')

row = df.first()

powerbi_json = [
    {
        'TotalVehicles': row['TotalVehicles'],
        'VehiclesOnTheWay': row['VehiclesOnTheWay'],
        'Timestamp': datetime.now().isoformat()
    }
]
resp = requests.post(powerbi_url, json=powerbi_json)
print(resp)

<Response [200]>